### Steps to get similar/duplicate questions

1. Preprocess data - clean question title and body.

remove html, punctuations, stopwords, words less than 4 chars,
but need to keep words like SQL, C++, C#... (get all tags list from tagsDF and exclude these from removing)

2. Apply TF-IDF count vectorizer and create sparse matrix.

3. Use NearestNeighbors and get most similar/duplicate 5 questions.

Challenges - keeping words like SQL, C#, C++ ....

For nearest neighbors, change metrics (mostly need to use cosine metric)

How to relate title and body ??

Can we use title , body and tag together to identify similar questions? Are tags reliable?

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer

In [18]:
questionsDF = pd.read_csv("Questions.csv", encoding='ISO-8859-1')
answersDF = pd.read_csv("Answers.csv", encoding='ISO-8859-1')
tagsDF = pd.read_csv("Tags.csv", encoding='ISO-8859-1')

In [180]:
questionsDF.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')

In [115]:
available_tags = tagsDF.Tag.unique()

In [127]:
html_removed_description = questionsDF['Body'].head(10000).apply(lambda x: BeautifulSoup(x,"html.parser").get_text()) 

In [134]:
html_removed_description[0]

"I've written a database generation script in SQL and want to execute it in my Adobe AIR application:\nCreate Table tRole (\n      roleID integer Primary Key\n      ,roleName varchar(40)\n);\nCreate Table tFile (\n    fileID integer Primary Key\n    ,fileName varchar(50)\n    ,fileDescription varchar(500)\n    ,thumbnailID integer\n    ,fileFormatID integer\n    ,categoryID integer\n    ,isFavorite boolean\n    ,dateAdded date\n    ,globalAccessCount integer\n    ,lastAccessTime date\n    ,downloadComplete boolean\n    ,isNew boolean\n    ,isSpotlight boolean\n    ,duration varchar(30)\n);\nCreate Table tCategory (\n    categoryID integer Primary Key\n    ,categoryName varchar(50)\n    ,parent_categoryID integer\n);\n...\n\nI execute this in Adobe AIR using the following methods:\npublic static function RunSqlFromFile(fileName:String):void {\n    var file:File = File.applicationDirectory.resolvePath(fileName);\n    var stream:FileStream = new FileStream();\n    stream.open(file, FileMo

In [138]:
###Preprocessing logic:

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\t", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

def clean_punct(text): 
    text = clean_text(text)
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    #remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in available_tags:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))




# # Function that takes text, tokenizes it and 
# # returns list of stemmed tokens
# def tokenize_and_stem(text):
#     tokens = nltk.word_tokenize(text)
#     stemmer = nltk.stem.porter.PorterStemmer()
#     return [i for i in [stemmer.stem(t) for t in tokens]

            
token = ToktokTokenizer()
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'
test=html_removed_description.apply(lambda x: clean_punct(x))           


In [146]:
test[0]

'i have written a database generation script in sql and want to execute it in my adobe air application create table trole roleid integer primary key rolename varchar 40 create table tfile fileid integer primary key filename varchar 50 filedescription varchar 500 thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean duration varchar 30 create table tcategory categoryid integer primary key categoryname varchar 50 parentcategoryid integer i execute this in adobe air using the following methods public static function runsqlfromfile filename string void var file file fileapplicationdirectoryresolvepath filename var stream filestream new filestream streamopen file filemoderead var strsql string streamreadutfbytes streambytesavailable nonquery strsql public static function nonquery strsql string void var sqlconnection sqlconnection new sqlconnection 

In [3]:
#questionsDF = questionsDF.head(1000)

In [5]:
#test = questionsDF['Body'].head(1).apply(lambda x: BeautifulSoup(x).get_text()) 

In [ ]:
#pd.value_counts(tagsDF['Tag'].head(1000)).plot(kind='bar')

In [7]:
#pd.Series(' '.join(tagsDF['Tag']).split())

In [241]:
# tagsDF['Tag'] = tagsDF['Tag'].astype(str)
# grouped_tags = tagsDF.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
# grouped_tags.reset_index()
# grouped_tags_final = pd.DataFrame({'Id':grouped_tags.index, 'Tags':grouped_tags.values})
# #questionsDF.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
# df = questionsDF.merge(grouped_tags_final, on='Id')
# new_df = df[df['Score']>5]
# new_df.drop(columns=['Id', 'Score'], inplace=True)
# new_df['Tags'] = new_df['Tags'].apply(lambda x: x.split())


# # new_df = tagsDF


In [101]:
flat_list = [item for sublist in new_df['Tags'].values for item in sublist]

In [35]:
test = questionsDF['Body']
#test = questionsDF['Body'].head(10000).apply(lambda x: BeautifulSoup(x).get_text()) 

In [218]:
x_train = test.str.join(' ')
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
x_train = vectorizer.fit_transform(test)


In [175]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(n_neighbors=5,metric="cosine").fit(x_train)
distances, indices = model_knn.kneighbors(x_train)

In [176]:
indices, distances

(array([[   0, 7678, 9845, 3782, 3643],
        [   1,  748, 3803,  499, 8404],
        [   2, 1238,  654, 6745, 7270],
        ...,
        [9997, 4309, 6645,  173, 1704],
        [9998, 9422, 8281, 5405, 3723],
        [9999, 5234, 4187, 5098,  899]]),
 array([[5.55111512e-16, 8.89156550e-01, 9.22372041e-01, 9.31548284e-01,
         9.36552946e-01],
        [0.00000000e+00, 9.47101622e-01, 9.56006900e-01, 9.57363426e-01,
         9.59523915e-01],
        [0.00000000e+00, 9.33556088e-01, 9.44787577e-01, 9.49644274e-01,
         9.49688322e-01],
        ...,
        [1.33226763e-15, 9.33453492e-01, 9.50300391e-01, 9.51046018e-01,
         9.52026899e-01],
        [0.00000000e+00, 8.90967885e-01, 9.38086153e-01, 9.56561520e-01,
         9.57508517e-01],
        [2.66453526e-15, 9.21539886e-01, 9.41754465e-01, 9.42523421e-01,
         9.49348578e-01]]))

In [190]:
for val in indices[7]:
    print(questionsDF["Title"][val] + "\n")


Deploying SQL Server Databases from Test to Live

Best practices for maintaining shared hosting websites with ASP.NET and SQL Server?

Best practice for graceful replace of read-only SQL Server database

Trigger to update data on another sql server

SQL Server Enterprise Manager 2005 - stored procedures not showing up



In [185]:
for val in indices[2]:
    print(test[val] + "\n")

has anyone got experience creating sql-based asp.net site-map providers i have got the default xml file web.sitemap working properly with my menu and sitemappath controls but i will need a way for the users of my site to create and modify pages dynamically i need to tie page viewing permissions into the standard asp.net membership system as well

is it possible to use an asp.net web.sitemap with a jquery superfish menu if not are there any standards based browser agnostic plugins available that work with the web.sitemap file

i am creating an asp.net web site where all pages hang off a database-driven tree-hierarchy pages typically present html content but some will execute programming examples 1 a contact us form or 2 a report generator how should i representreference the programming within the database should i have a varchar value of a web user control ascx name or a web form aspx name something else or should it just be an integer or other such id in a dictionary within my applicat

In [191]:
questionsDF["Body"][20000]

'<p>I am trying to get my application an installer via WiX 3.0. The exact code is:</p>\n\n<p></p>\n\n<pre><code>&lt;File Id="ServiceComponentMain" Name="$(var.myProgramService.TargetFileName)" Source="$(var.myProgramService.TargetPath)" DiskId="1" Vital="yes"/&gt;\n\n&lt;!-- service will need to be installed under Local Service --&gt;\n&lt;ServiceInstall\n\t\t\t\t\t\tId="MyProgramServiceInstaller"\n\t\t\t\t\t\tType="ownProcess"\n\t\t\t\t\t\tVital="yes"\n\t\t\t\t\t\tName="MyProgramAddon"\n\t\t\t\t\t\tDisplayName="[removed]"\n\t\t\t\t\t\tDescription="[removed]"\n\t\t\t\t\t\tStart="auto"\n\t\t\t\t\t\tAccount="LocalService"\n\t\t\t\t\t\tErrorControl="ignore"\n\t\t\t\t\t\tInteractive="no"/&gt;\n&lt;ServiceControl Id="StartDDService" Name="MyProgramServiceInstaller" Start="install" Wait="no" /&gt;\n&lt;ServiceControl Id="StopDDService" Name="MyProgramServiceInstaller" Stop="both" Wait="yes" Remove="uninstall" /&gt;\n</code></pre>\n\n<p></p>\n\n<p>Thing is, for some reason LocalService fails 

In [192]:
question = '<p>I am trying to get my application an installer via WiX 3.0. The exact code is:</p>\n\n<p></p>\n\n<pre><code>&lt;File Id="ServiceComponentMain" Name="$(var.myProgramService.TargetFileName)" Source="$(var.myProgramService.TargetPath)" DiskId="1" Vital="yes"/&gt;\n\n&lt;!-- service will need to be installed under Local Service --&gt;\n&lt;ServiceInstall\n\t\t\t\t\t\tId="MyProgramServiceInstaller"\n\t\t\t\t\t\tType="ownProcess"\n\t\t\t\t\t\tVital="yes"\n\t\t\t\t\t\tName="MyProgramAddon"\n\t\t\t\t\t\tDisplayName="[removed]"\n\t\t\t\t\t\tDescription="[removed]"\n\t\t\t\t\t\tStart="auto"\n\t\t\t\t\t\tAccount="LocalService"\n\t\t\t\t\t\tErrorControl="ignore"\n\t\t\t\t\t\tInteractive="no"/&gt;\n&lt;ServiceControl Id="StartDDService" Name="MyProgramServiceInstaller" Start="install" Wait="no" /&gt;\n&lt;ServiceControl Id="StopDDService" Name="MyProgramServiceInstaller" Stop="both" Wait="yes" Remove="uninstall" /&gt;\n</code></pre>\n\n<p></p>\n\n<p>Thing is, for some reason LocalService fails on the "Installing services" step, and if I change it to "LocalSystem" then the installer times out while trying to start the service.</p>\n\n<p>The service starts fine manually and at system startup, and for all intents and purposes works great. I\'ve heard there are issues getting services to work right under LocalService, but Google isnt really helping as everyone\'s responses have been "got it to work kthx".</p>\n\n<p>Just looking to get this service set up and started during installation, that\'s all. Any help? Thanks!</p>\n'

In [193]:
question_cleaned = BeautifulSoup(question,"html.parser").get_text()
question_cleaned = clean_punct(question_cleaned)

In [194]:
question_cleaned

'i am trying to get my application an installer via wix 30 the exact code is file id servicecomponentmain name varmyprogramservicetargetfilename source varmyprogramservicetargetpath diskid 1 vital yes -- service will need to be installed under local service -- serviceinstall id myprogramserviceinstaller type ownprocess vital yes name myprogramaddon displayname removed description removed start auto account localservice errorcontrol ignore interactive no servicecontrol id startddservice name myprogramserviceinstaller start install wait no servicecontrol id stopddservice name myprogramserviceinstaller stop both wait yes remove uninstall thing is for some reason localservice fails on the installing services step and if i change it to localsystem then the installer times out while trying to start the service the service starts fine manually and at system startup and for all intents and purposes works great i have heard there are issues getting services to work right under localservice but 

In [162]:
question_cleaned = clean_punct(question_cleaned)
question_cleaned

'i am trying to get my application an installer via wix 30 the exact code is file id servicecomponentmain name varmyprogramservicetargetfilename source varmyprogramservicetargetpath diskid 1 vital yes -- service will need to be installed under local service -- serviceinstall id myprogramserviceinstaller type ownprocess vital yes name myprogramaddon displayname removed description removed start auto account localservice errorcontrol ignore interactive no servicecontrol id startddservice name myprogramserviceinstaller start install wait no servicecontrol id stopddservice name myprogramserviceinstaller stop both wait yes remove uninstall thing is for some reason localservice fails on the installing services step and if i change it to localsystem then the installer times out while trying to start the service the service starts fine manually and at system startup and for all intents and purposes works great i have heard there are issues getting services to work right under localservice but 

In [221]:
testVector

<1x1240371 sparse matrix of type '<class 'numpy.float64'>'
	with 199 stored elements in Compressed Sparse Row format>

In [260]:
similar_question1 = ""

In [284]:
def pred_similar(question):
    question_cleaned = BeautifulSoup(question,"html.parser").get_text()
    question_cleaned = clean_punct(question_cleaned)

    testVector = vectorizer.transform(pd.Series(question_cleaned))
    from sklearn.neighbors import NearestNeighbors
    from sklearn.metrics.pairwise import cosine_similarity
    # #model_knn = NearestNeighbors(n_neighbors=5,metric="cosine").fit(question_cleaned)
    # distances, indices = model_knn.kneighbors(testVector)
    #testVector
    sim = cosine_similarity(testVector,x_train)
    
    temp = []
    # now create dictionary adding indices to it
    inds = np.arange(0,10000)
    for row in sim: 
        temp.append(list(zip(inds,row)))

    sortedSims = []
    for row in temp:
        row.sort(key = lambda kv:kv[1], reverse=True)
        sortedSims.append(row)
        
    
    five_similar = []
    for i in range(0,5):
        five_similar.append(test[sortedSims[0][i][0]])
    #similar_question1 = test[sortedSims[0][0][0]]
    
    #return similar_question1
    
    return five_similar

        
    

        

In [282]:
test[sortedSims[0][2][0]]


'suppose some windows service uses code that wants mapped network drives and no unc paths how can i make the drive mapping available to the service session when the service is started logging in as the service user and creating a persistent mapping will not establish the mapping in the context of the actual service'

In [283]:
sortedSims[0][2][0]

2278

In [237]:
question_cleaned

'i am trying to get my application an installer via wix 30 the exact code is file id servicecomponentmain name varmyprogramservicetargetfilename source varmyprogramservicetargetpath diskid 1 vital yes -- service will need to be installed under local service -- serviceinstall id myprogramserviceinstaller type ownprocess vital yes name myprogramaddon displayname removed description removed start auto account localservice errorcontrol ignore interactive no servicecontrol id startddservice name myprogramserviceinstaller start install wait no servicecontrol id stopddservice name myprogramserviceinstaller stop both wait yes remove uninstall thing is for some reason localservice fails on the installing services step and if i change it to localsystem then the installer times out while trying to start the service the service starts fine manually and at system startup and for all intents and purposes works great i have heard there are issues getting services to work right under localservice but 

In [235]:
test[7557]

'i am curious to know how i would go about setting up my service to stop cleanly on the server the service will be installed on for example when i have many clients connecting and doing operations every minute and i want to shut-down the service for maintenance how can i do this in the onstop event of the service to then let the main service host to deny any new client connections and let the current connections finish before it actually shuts down its services to the client this will ensure data is not corrupted on the server as the server shuts down right now i am not setup as a singleton because i need scalability in the service so i would have to somehow get my service host to do this independently of knowing how many instances are created of the service class'

In [236]:
test[5320]

'i have upgraded a vdproj msi generator project built into vs2008 systemconfigurationinstallinstaller with a serviceprocessinstaller and a serviceinstaller from visual studio 2005 to 2008 there are no customisations of consequence to the installer class ie not trying to start or stop services or register children removepreviousversions is set to true and i am changing the version and productcode this triggers an error during the install error 1001 the specified service already exists googling yields stuff but not on so until now - google for the specified service already exists the most useful one i have seen to date is httpforumsmsdnmicrosoftcomen-uswinformssetupthreadb2d1bd22-8499-454e-9cec-1e42c03e2557 however this does not answer the fundamental question - given that the user can pick either a an install location that the same or b an install location that different what are the minimal code changes would one sensibly make to ensure that the old service gets uninstalled and the new

In [229]:
sorted(sim[0], reverse=True)

[0.11121889641490781,
 0.10021907147170188,
 0.08406052838272723,
 0.08298187362081345,
 0.08138773125970196,
 0.08117151593873023,
 0.0806527341043857,
 0.07992496149062916,
 0.0788758484530438,
 0.07605234868709415,
 0.07401154590010509,
 0.07374205274342484,
 0.07272020680011826,
 0.07261075853349101,
 0.07194539549345623,
 0.07171207736520586,
 0.07020666067324559,
 0.06945843346108757,
 0.06844286004731019,
 0.06766942900676673,
 0.06690409869155274,
 0.06689438579351306,
 0.06666086956054787,
 0.06641590456641933,
 0.06613609845200942,
 0.06562680737301538,
 0.0655766089408491,
 0.06537166869620531,
 0.06525242796598806,
 0.06484096197436943,
 0.06455855261105445,
 0.06341733046885645,
 0.06271694491364832,
 0.062070929134545975,
 0.06201847943912907,
 0.0619530167979797,
 0.06187838089472274,
 0.061204356312591635,
 0.06120272784008322,
 0.05996274639307887,
 0.059895079591724174,
 0.05987452206933461,
 0.05971158714343336,
 0.059644725748163055,
 0.05962311277687806,
 0.0594849

In [ ]:
#train = questionsDF["Body"]

In [ ]:
# from IPython.display import Audio
# sound_file = "bottle_pop_2.wav"
# Audio(filename=sound_file, autoplay=True)

In [ ]:
# def remove_punctuation(words):
#     """Remove punctuation from list of tokenized words"""
    
#     new_words = []
#     for word in words:
#         new_word = re.sub(r'[^\w\s]', '', word)
#         if new_word != '':
#             new_words.append(new_word)
#     return new_words



# def lemmatize(words):
#     text = " ".join(WordNetLemmatizer().lemmatize(word) for word in words.split())
#     return text.split()

# def remove_less4(words):
#     text = re.sub(r'\b\w{1,3}\b', '', words)
#     return text

# def preprocess_data(text):  
#     text = text.lower()
#     text = re.sub(r'\b\w{1,3}\b', '', text)   #remove words less than 3
#     text = re.sub(r'[^\w\s]','',text)
#     text = re.sub(r'\d+', '', text)
#     return text





# train = train.apply(lambda x: " ".join(x.lower() for x in x.split()))
# train = train.str.replace('[^\w\s]','')
# stop = stopwords.words('english')
# train = train.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# freq = pd.Series(' '.join(train[1]).split()).value_counts()[:10]
# freq = list(freq.index)
# train[1] = train[1].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# freq = pd.Series(' '.join(train[1]).split()).value_counts()[-10:]
# freq = list(freq.index)
# train[1] = train[1].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# from textblob import TextBlob
# TextBlob(train[1][1]).words
# from textblob import Word
# train[1] = train[1].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


# x_train = train[1]
# #x_train = train[1].apply(preprocess_data)
# #train[1] = train[1].apply(lambda x: " ".join(x.lower() for x in x.split()))
# #freq = pd.Series(' '.join(x_train).split()).value_counts()[:30]
# #x_train = x_train.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# # from IPython.display import Audio
# # sound_file = "bottle_pop_2.wav"
# # Audio(filename=sound_file, autoplay=True)
# # x_train_unVectorized = x_train
# #x_train = x_train.str.join(' ')

In [ ]:
#questionsDF.loc[questionsDF['Id'] == 80]
#answersDF.loc[answersDF['ParentId'] == 80]
import pickle
filename = 'similar_questions_model'
outfile = open(filename,'wb')
pickle.dump(similar_questions_model,outfile)
outfile.close()

In [142]:
import numpy as np
from sklearn.metrics import hamming_loss
hamming_loss(np.array([[0, 1], [1, 1]]), np.zeros((2, 2)))

0.75

In [144]:
np.array([[0, 1], [1, 1]])

array([[0, 1],
       [1, 1]])

In [145]:
np.zeros((2, 2))

array([[0., 0.],
       [0., 0.]])

In [288]:
from flask import Flask
from flask import request
from flask import render_template
app = Flask(__name__, template_folder = 'templates')

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    else:
        #tagg = request.form['tag']   
        question = request.form.get('tagg')
        #similar_quest = "Asas"
        similar_quest = pred_similar(question)
        #similar_questions = get_similar()
        return render_template('index.html', similarquestion_1=similar_quest[0], similarquestion_2=similar_quest[1], similarquestion_3=similar_quest[2], similarquestion_4=similar_quest[3], similarquestion_5=similar_quest[4])
   
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2019 22:12:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2019 22:12:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2019 22:13:05] "POST / HTTP/1.1" 200 -
